In [ ]:
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        Tool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. Use this tool by sending a pair of number seperated by a comma.\nExample:1,2",
        ),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

In [7]:
import os
import requests
from typing import Type
from langchain_openai import ChatOpenAI
from langchain.tools import Tool, BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1)

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockSymbolSearchToolArgsScheme(BaseModel):
    query: str = Field(description="The query you will search for")


class StockSymbolSearchTool(BaseTool):
    name = "StockSymbolSearchTool"
    description = """
    Use this tool to find the stock symbol for a company. 
    It takes a query as an argument. 
    Example querty: Stock symbol for the Apple Company
    """
    args_schema: Type[StockSymbolSearchToolArgsScheme] = StockSymbolSearchToolArgsScheme

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsScheme(BaseModel):
    symbol: str = Field(description="Stock symbol of the compadny. Example:APPL, TSLA")


class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsScheme] = CompanyOverviewArgsScheme

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsScheme] = CompanyOverviewArgsScheme

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]


class CompanyStockHistoryTool(BaseTool):
    name = "CompanyStockHistory"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsScheme] = CompanyOverviewArgsScheme

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        response = r.json()
        return list(response["Weekly Time Series"].items())[:200]


agent = initialize_agent(
    llm=llm,
    # verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockHistoryTool(),
    ],
)

prompt = "Give me information on Rivian's stock, considering its fiancials, income statements, and stock performance, help me analyze if it's a potentially good investment. Also tell me what stock symbol this stock has."

result = agent.invoke(prompt)
print(result["output"])

### Rivian Stock Analysis:

#### Stock Symbol: RIVN

#### Company Overview:
- **Sector:** Manufacturing
- **Industry:** Motor Vehicles & Passenger Car Bodies
- **Address:** 14600 MYFORD ROAD, IRVINE, CA, US
- **Market Capitalization:** $14,671,017,000
- **Revenue (TTM):** $4,977,000,000
- **Gross Profit (TTM):** -$3,123,000,000
- **Net Income (TTM):** -$5,432,000,000
- **EPS (TTM):** -$5.77
- **Profit Margin:** -1.11%
- **Operating Margin (TTM):** -1.23%
- **Return on Assets (TTM):** -0.21%
- **Return on Equity (TTM):** -0.53%
- **52-Week High:** $28.06
- **52-Week Low:** $8.26

#### Income Statement:
- **2023 Revenue:** $4,214,000,000
- **2023 Gross Profit:** -$2,030,000,000
- **2023 Operating Income:** -$5,739,000,000
- **2023 Net Income:** -$5,432,000,000

#### Stock Performance:
- **Current Price:** $14.74
- **52-Week High:** $28.06
- **52-Week Low:** $8.26
- **50-Day Moving Average:** $10.39
- **200-Day Moving Average:** $15.19

### Analysis:
- Rivian has shown a negative profit m

In [3]:
d = {
    "Weekly Time Series": {
        "x1": 1,
        "x2": 2,
        "x3": 3,
    }
}

list(d["Weekly Time Series"].items())[:2]

[('x1', 1), ('x2', 2)]

In [12]:
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.agents import create_sql_agent, AgentType

llm = ChatOpenAI(temperature=0)
db = SQLDatabase.from_uri("sqlite:///movies.sqlite")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    # verbose=True,
)

response = agent.invoke("Give me 5 directors that have the highest grossing films.")
print(response["output"])

The 5 directors with the highest grossing films are director_id 4799, 4777, 4762, 4788, and 4765.


In [19]:
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase

llm = ChatOpenAI(temperature=0)
db = SQLDatabase.from_uri("sqlite:///movies.sqlite")
agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type="openai-tools",
    verbose=True,
)

print(toolkit.get_tools())

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10fe7bdd0>), InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10fe7bdd0>), ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10fe7bdd0>), QuerySQLCheckerTool(description='Use this tool to double check if your que